In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 45.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=cf19a3f63ee9e48a515e56a3a343b2173044d6525bf570c7d1050be3b069a192
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


***Przewidywanie wzięcia pożyczki. ***

Celem projektu jest zastosowanie modeli  klasyfikacji (binarnej) do  przewidzenia czy pożyczka zostanie udzielona danemu klientowi z danymi parametrami wniosku. 

Zmienna celu (wypłacenie pożyczki): ***Disbursed ***


---



***Wymogi:*** 

● Zmienna LoggedIn nie może być uwzględniona w modelowaniu - należy ją od razu wyrzucić ze  zbioru.  
● Nie usuwamy żadnych obserwacji - braki danych uzupełniamy w jakiś sposób.  
● Wszelkie nieoczywiste ​ przekształcenia danych należy opatrzyć uzasadnieniem​ dlaczego  dokonujemy tego przekształcenia (np. robimy wykres i uzasadniamy wzięci logarytmu ze  zmiennej tym, że rozkład jest skośny).  
● Należy wykorzystać przynajmniej 3 metody klasyfikacji.  
● Należy uwzględnić optymalizację modeli/pipelinów.  
● Projekt musi być zakończony przejrzystym porównaniem przetestowanych rozwiązań  (modeli/pipelinów) ​ w postaci tabeli​ , gdzie jeden wiersz opisuje jedno rozwiązanie i jego  wynik. 



Link do danych:  https://raw.githubusercontent.com/saimadhu-polamuri/DataHakthon3X/master/dataSet/Train.csv 


Opis danych (najdokładniejszy jaki istnieje):  https://discuss.analyticsvidhya.com/t/hackathon-3-x-predict-customer-worth-for-happy-customer-bank/3802 

**IMPORT BIBLIOTEK**

In [2]:
import pyspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName('my_app').getOrCreate()

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import feature

In [5]:
spark

***Import Danych, Przekształcenia***

In [6]:
csv_df = spark.read.csv('/content/Train.csv', header=True, inferSchema=True)


In [7]:

csv_df.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+--------+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|LoggedIn|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------

Wyrzucenie ze zbioru zmiennej **LoggedIn**

In [8]:
tmp = csv_df.drop('LoggedIn')
tmp.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+

In [9]:
tmp.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = true)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = true)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = true)
 |-- Filled_Form: string (nullable = true)
 |-- Device_Type: string (nullable = true)
 |-- Var2: string (nullable = true)
 |-- Source: string (nul

In [10]:
tmp.describe().show()

+-------+-----------+------+--------------+------------------+---------+------------------+-------------------+-------------------+-----------------+--------------------+--------------------+---------------+-----------------+-----+---------------------+---------------------+------------------+------------------+------------------+-----------+-----------+-----+------+------------------+-------------------+
|summary|         ID|Gender|          City|    Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|     Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|             Var5| Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|     Interest_Rate|    Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type| Var2|Source|              Var4|          Disbursed|
+-------+-----------+------+--------------+------------------+---------+------------------+-------------------+-------------------+-----------------+--------------------+------------

In [11]:
# zmieniamy Null na 0

tmp = tmp.fillna(0)
tmp.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+

In [12]:
tmp.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = false)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = false)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = false)
 |-- Filled_Form: string (nullable = true)
 |-- Device_Type: string (nullable = true)
 |-- Var2: string (nullable = true)
 |-- Source: string (

In [13]:
tmp.groupBy("Disbursed").count().show()

+---------+-----+
|Disbursed|count|
+---------+-----+
|        1| 1273|
|        0|85747|
+---------+-----+



In [14]:
#tmp.toPandas()


In [15]:
# Liczba pożyczek
tmp.select((tmp["Disbursed"]).alias("cln")).describe().show()

+-------+-------------------+
|summary|                cln|
+-------+-------------------+
|  count|              87020|
|   mean|0.01462882096069869|
| stddev|0.12006241797245076|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [16]:
tmp.createOrReplaceTempView("tmp")

In [17]:
# max/min/avg udzielonej pożyczki

spark.sql('select max(Loan_Amount_Applied) as Max_Loan_Amount, min(Loan_Amount_Applied) as Min_Loan_Amount, avg(Loan_Amount_Applied) as Avg_Loan_Amout \
      from tmp \
      where Disbursed == 1').show()

+---------------+---------------+-----------------+
|Max_Loan_Amount|Min_Loan_Amount|   Avg_Loan_Amout|
+---------------+---------------+-----------------+
|        3000000|              0|325259.2482325216|
+---------------+---------------+-----------------+



In [18]:
# sprawdzenie unikalności danych
tmp.groupBy("Device_Type").count().show()
tmp.groupBy("Gender").count().show()
tmp.groupBy("Filled_Form").count().show()
tmp.groupBy("Source").count().show()
tmp.groupBy("Mobile_Verified").count().show()

+-----------+-----+
|Device_Type|count|
+-----------+-----+
|Web-browser|64316|
|     Mobile|22704|
+-----------+-----+

+------+-----+
|Gender|count|
+------+-----+
|Female|37172|
|  Male|49848|
+------+-----+

+-----------+-----+
|Filled_Form|count|
+-----------+-----+
|          Y|19490|
|          N|67530|
+-----------+-----+

+------+-----+
|Source|count|
+------+-----+
|  S155|    4|
|  S143| 4332|
|  S158|  208|
|  S151|  720|
|  S162|   36|
|  S136|    3|
|  S157|  650|
|  S160|   11|
|  S159| 5599|
|  S161|  769|
|  S129|    3|
|  S135|    2|
|  S125|    1|
|  S156|  308|
|  S154|    1|
|  S150|   10|
|  S138|    3|
|  S122|38567|
|  S133|29885|
|  S123|   73|
+------+-----+
only showing top 20 rows

+---------------+-----+
|Mobile_Verified|count|
+---------------+-----+
|              Y|56481|
|              N|30539|
+---------------+-----+



Konwersja danych

In [19]:
from pyspark.sql.functions import col, when

In [20]:
tmp1 = tmp.select(col("*"), when(col("Gender") == "Male",1)
      .when(col("Gender") == "Female",0).alias("new_gender"), when(col("Filled_Form") == "Y",1)
      .when(col("Filled_Form") == "N",0).alias("new_Filled_Form"), when(col("Device_Type") == "Web-browser",1)
      .when(col("Device_Type") == "Mobile",0).alias("new_Device_Type"), when(col("Mobile_Verified") == "Y",1)
      .when(col("Mobile_Verified") == "N",0).alias("new_Mobile_Verified"))
tmp1.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+----------+---------------+---------------+-------------------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+

In [22]:
tmp2 = tmp1.drop("Gender", "Mobile_Veified", "Device_Type", "ID", "Filled_Form")
tmp2.show()


+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+---

In [23]:
tmp2.select("Employer_Name", "Disbursed").groupBy("Employer_Name").sum().orderBy(f.desc("sum(Disbursed)")).show()

+--------------------+--------------+
|       Employer_Name|sum(Disbursed)|
+--------------------+--------------+
|TATA CONSULTANCY ...|            21|
|ACCENTURE SERVICE...|            12|
|COGNIZANT TECHNOL...|            10|
|      INDIAN RAILWAY|             7|
|HCL TECHNOLOGIES LTD|             7|
|             GENPACT|             7|
|           CIPLA LTD|             5|
|              OTHERS|             5|
|  HCL AVITAS PVT LTD|             5|
|IBM GLOBAL SERVIC...|             5|
|TCS AND ASSOCIATE...|             5|
|                   0|             5|
|     IBM CORPORATION|             4|
|ERICSSON INDIA GL...|             4|
|                BSNL|             4|
|HCL - HEWLETT-PAC...|             4|
|CAPGEMINI INDIA P...|             4|
|   IDEA CELLULAR LTD|             4|
|TYPE SLOWLY FOR A...|             4|
|HSBC ELECTRONIC D...|             4|
+--------------------+--------------+
only showing top 20 rows



In [24]:
tmp2 = tmp2.fillna(0)
tmp2.show()

+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+---

In [25]:
df_t, df_e = tmp2.randomSplit([0.7, 0.3], 42)

In [26]:
df_t.show()

+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+-------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|    City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|     Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+-------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---

In [27]:
tmp2.printSchema()

root
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = false)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = false)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = false)
 |-- Var2: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Var4: integer (nullable = true)
 |-- Disbursed: integer (nullable = true)
 |-- new_gender: integer (nullable = true)
 |-- new_Filled_For

In [28]:
tmp2.show(3, vertical=True)

-RECORD 0-------------------------------------
 City                  | Delhi                
 Monthly_Income        | 20000                
 DOB                   | 23-May-78            
 Lead_Creation_Date    | 15-May-15            
 Loan_Amount_Applied   | 300000               
 Loan_Tenure_Applied   | 5                    
 Existing_EMI          | 0.0                  
 Employer_Name         | CYBOSOL              
 Salary_Account        | HDFC Bank            
 Mobile_Verified       | N                    
 Var5                  | 0                    
 Var1                  | HBXX                 
 Loan_Amount_Submitted | 0                    
 Loan_Tenure_Submitted | 0                    
 Interest_Rate         | 0.0                  
 Processing_Fee        | 0                    
 EMI_Loan_Submitted    | 0.0                  
 Var2                  | G                    
 Source                | S122                 
 Var4                  | 1                    
 Disbursed   

In [29]:
formula = ("Disbursed ~ new_gender + new_Device_Type + new_Mobile_Verified + Var1 + new_Filled_Form + Var2 + Source +"
                              "Monthly_Income + Loan_Amount_Applied + Loan_Tenure_Applied")
rf = feature.RFormula(formula=formula, featuresCol='featuresFull')

In [30]:
rfModel = rf.fit(tmp2)

In [31]:
tmp3 = rfModel.transform(tmp2)
tmp3.show(truncate=False)

+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------------------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+-----------------------------------------------------------------------------+-----+
|City       |Monthly_Income|DOB      |Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|Employer_Name                               |Salary_Account      |Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|featuresFull                                                                 |label|
+-----------+--------------+---------+------------------+-----

In [32]:
#Skalowanie
scaler = feature.StandardScaler(inputCol="featuresFull", outputCol="featuresScal")

In [33]:
scalerModel = scaler.fit(tmp3)

In [34]:
tmp3.select("featuresFull").show(truncate=False)
scalerModel.transform(tmp3).select("featuresScal").show(truncate=False)

+-----------------------------------------------------------------------------+
|featuresFull                                                                 |
+-----------------------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |
|(60,[0,1,2,7,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|
|(60,[0,1,2,3,22,34,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,100000.0,500000.0,2.0])|
|(60,[0,1,2,15,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,45000.0,300000.0,5.0])|
|(60,[1,3,22,29,57,58,59],[1.0,1.0,1.0,1.0,70000.0,6.0,5.0])                  |
|(60,[0,1,2,3,22,30,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,20000.0,200000.0,5.0]) |
|(60,[0,2,8,21,24,28,57],[1.0,1.0,1.0,1.0,1.0,1.0,75000.0])                   |
|(60,[1,2,4,22,29,57,58,59],[1.0,1.0,1.0

In [35]:
MMscaler = feature.MinMaxScaler(inputCol="featuresFull", outputCol="featuresScal")

In [36]:
MMscalerModel = MMscaler.fit(tmp3)

In [37]:
tmp3.select("featuresFull").show(truncate=False)
MMscalerModel.transform(tmp3).select("featuresScal").show(truncate=False)

+-----------------------------------------------------------------------------+
|featuresFull                                                                 |
+-----------------------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |
|(60,[0,1,2,7,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|
|(60,[0,1,2,3,22,34,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,100000.0,500000.0,2.0])|
|(60,[0,1,2,15,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,45000.0,300000.0,5.0])|
|(60,[1,3,22,29,57,58,59],[1.0,1.0,1.0,1.0,70000.0,6.0,5.0])                  |
|(60,[0,1,2,3,22,30,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,20000.0,200000.0,5.0]) |
|(60,[0,2,8,21,24,28,57],[1.0,1.0,1.0,1.0,1.0,1.0,75000.0])                   |
|(60,[1,2,4,22,29,57,58,59],[1.0,1.0,1.0

**PCA**

In [38]:
pca = feature.PCA(k=3, inputCol="featuresFull", outputCol="featuresPCA")

In [39]:
PCAmodel = pca.fit(tmp3)

In [40]:
PCAmodel.transform(tmp3).select("featuresFull", "featuresPCA").show(truncate=False)

+-----------------------------------------------------------------------------+------------------------------------------------------------+
|featuresFull                                                                 |featuresPCA                                                 |
+-----------------------------------------------------------------------------+------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |[-20152.98625505873,-299989.76174707466,4.097771557942352]  |
|(60,[0,1,2,7,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |[-35101.98801937641,-199982.12529907905,1.3787087613486313] |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |[-22805.974785690658,-599988.4478268801,2.339282178163524]  |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|[-35509.95830123298,-999982.0212799263,2.14316066116888]    |
|(60,[0,1,2,3

In [41]:
PCAmodel.explainedVariance

DenseVector([0.9742, 0.0258, 0.0])

In [42]:
rfModel = rf.fit(tmp2)

In [43]:
df_train = rfModel.transform(df_t)
df_eval = rfModel.transform(df_e)

In [44]:
scaler = feature.StandardScaler(inputCol="featuresFull", outputCol="features")
scal_mod = scaler.fit(df_train)
df_train = scal_mod.transform(df_train)
df_eval = scal_mod.transform(df_eval)

df_eval.show()

+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+--------------------+-----+--------------------+
|    City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|        featuresFull|label|            features|
+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+-------------

In [45]:
num_cols = [c for c, t in df_t.dtypes if t != "string"]
categ_cols = [c for c, t in df_t.dtypes if t == "string" and c != "Disbursed"]
categ_cols_idx = [c + "Idx" for c in categ_cols]
categ_cols_vect = [c + "Vect" for c in categ_cols]

In [46]:
indexer = feature.StringIndexer(inputCol="Disbursed", outputCol="label1")
indexers = feature.StringIndexer(inputCols=categ_cols, outputCols=categ_cols_idx)
OHencoder = feature.OneHotEncoder(inputCols=categ_cols_idx, outputCols=categ_cols_vect)
vectAssembler = feature.VectorAssembler(inputCols = num_cols + categ_cols_vect, outputCol = "featuresRaw")
#scaler = feature.StandardScaler(inputCol="featuresRaw", outputCol="features")

In [47]:
pipe = Pipeline(stages=[indexer, indexers, OHencoder, vectAssembler, scaler])
pipeModel = pipe.fit(tmp3)

In [48]:
df_train = pipeModel.transform(tmp3)
df_eval = pipeModel.transform(tmp3)

In [ ]:
df_eval.show()

In [50]:
df_train = df_train.select("label", "features")
df_eval = df_eval.select("label", "features")

In [ ]:
print("Train:")
df_train.groupBy("label").count().show()
print("Eval:")
df_eval.groupBy("label").count().show()

In [52]:
df_train.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(60,[1,3,23,28,57...|
|  0.0|(60,[0,1,2,7,23,2...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,15,22,...|
|  0.0|(60,[1,3,22,29,57...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,2,8,21,24,...|
|  0.0|(60,[1,2,4,22,29,...|
|  0.0|(60,[0,1,2,4,22,2...|
|  0.0|(60,[1,3,22,28,57...|
|  0.0|(60,[1,3,22,29,57...|
|  0.0|(60,[0,1,2,9,22,2...|
|  0.0|(60,[1,3,22,36,57...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,11,25,...|
|  0.0|(60,[1,3,22,28,57...|
|  0.0|(60,[1,3,25,29,57...|
|  0.0|(60,[0,1,2,3,25,2...|
+-----+--------------------+
only showing top 20 rows



REGRESJA LINIOWA

In [53]:
from pyspark.ml import regression

In [54]:
reg = regression.LinearRegression(maxIter=500)

In [55]:
regModel = reg.fit(df_train)

In [56]:
regModel.coefficients

DenseVector([0.0031, 0.0, 0.0019, 0.0118, 0.0041, 0.002, 0.0047, 0.0075, 0.0089, 0.0049, 0.0081, 0.0015, 0.0028, 0.0012, 0.0016, 0.0023, 0.0007, 0.0006, 0.0002, 0.0008, 0.0002, 0.0037, 0.0029, -0.0026, 0.0021, 0.0001, -0.0016, -0.0002, 0.0043, -0.0001, -0.0016, 0.0006, -0.0013, -0.0041, 0.0022, -0.0013, -0.0011, -0.0008, -0.0007, -0.0013, -0.0006, -0.0006, -0.0007, -0.0, -0.0004, -0.0003, 0.0009, -0.0003, -0.0001, -0.0001, -0.0001, 0.0, -0.0001, -0.0002, 0.0, -0.0, -0.0, -0.0001, 0.0023, 0.0043])

In [57]:
regModel.intercept

-0.02854028698498677

In [58]:
trainSummary = regModel.summary
type(trainSummary)

pyspark.ml.regression.LinearRegressionTrainingSummary

In [59]:
trainSummary.meanAbsoluteError

0.029156022120266634

In [60]:
trainSummary.meanSquaredError

0.014211110364853312

In [61]:
trainSummary.r2

0.014131859677981695

In [62]:
# predykcje
regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.012673119870353938|
|  0.0|(60,[0,1,2,7,23,2...| 0.03894956139590243|
|  0.0|(60,[0,1,2,3,22,3...|0.028089817612023955|
|  0.0|(60,[0,1,2,3,22,3...|0.032857002053417704|
|  0.0|(60,[0,1,2,3,22,3...| 0.03824227434040331|
|  0.0|(60,[0,1,2,15,22,...| 0.04470125804214896|
|  0.0|(60,[1,3,22,29,57...|0.013093366614811745|
|  0.0|(60,[0,1,2,3,22,3...| 0.01820400922026639|
|  0.0|(60,[0,2,8,21,24,...| 0.06421188539978266|
|  0.0|(60,[1,2,4,22,29,...| 0.00300531945764259|
|  0.0|(60,[0,1,2,4,22,2...| 0.01808183427953043|
|  0.0|(60,[1,3,22,28,57...|0.020768572297168775|
|  0.0|(60,[1,3,22,29,57...|0.014294991168959222|
|  0.0|(60,[0,1,2,9,22,2...| 0.03232506263683215|
|  0.0|(60,[1,3,22,36,57...|-0.00690761697913...|
|  0.0|(60,[0,1,2,3,22,3...|0.015960034011154214|
|  0.0|(60,[0,1,2,11,25,...| 0.01868713240699233|


#### Drzewo regresyjne

In [63]:
tree_reg = regression.DecisionTreeRegressor()

In [64]:
tree_regModel = tree_reg.fit(df_train)

In [65]:
print(tree_regModel.toDebugString)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_6255ff4491b1, depth=5, numNodes=57, numFeatures=60
  If (feature 57 <= 0.016470407748875524)
   If (feature 57 <= 0.009189159861454218)
    If (feature 57 <= 0.008271828252805586)
     If (feature 5 <= 2.2628782783944144)
      If (feature 21 <= 1.1993123157773127)
       Predict: 0.0
      Else (feature 21 > 1.1993123157773127)
       Predict: 4.553734061930783E-4
     Else (feature 5 > 2.2628782783944144)
      If (feature 59 <= 0.7446789795243225)
       Predict: 0.0
      Else (feature 59 > 0.7446789795243225)
       Predict: 0.002081165452653486
    Else (feature 57 > 0.008271828252805586)
     If (feature 6 <= 2.7815593385283988)
      If (feature 28 <= 1.006510530627566)
       Predict: 0.0023059185242121443
      Else (feature 28 > 1.006510530627566)
       Predict: 0.004461221688400824
     Else (feature 6 > 2.7815593385283988)
      If (feature 0 <= 1.0107756116042783)
       Predict: 0.05263157894736842
      Else (featu

In [67]:
# predykcje
tree_regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.004461221688400824|
|  0.0|(60,[0,1,2,7,23,2...|0.013038968736109054|
|  0.0|(60,[0,1,2,3,22,3...|0.005582290664100096|
|  0.0|(60,[0,1,2,3,22,3...|0.007229612492770388|
|  0.0|(60,[0,1,2,3,22,3...| 0.04672897196261682|
|  0.0|(60,[0,1,2,15,22,...| 0.03868207103123663|
|  0.0|(60,[1,3,22,29,57...| 0.03868207103123663|
|  0.0|(60,[0,1,2,3,22,3...|0.002305918524212...|
|  0.0|(60,[0,2,8,21,24,...| 0.05109126984126984|
|  0.0|(60,[1,2,4,22,29,...|0.023845763571790968|
|  0.0|(60,[0,1,2,4,22,2...|0.005582290664100096|
|  0.0|(60,[1,3,22,28,57...|0.005582290664100096|
|  0.0|(60,[1,3,22,29,57...|0.005582290664100096|
|  0.0|(60,[0,1,2,9,22,2...|0.023845763571790968|
|  0.0|(60,[1,3,22,36,57...|0.013038968736109054|
|  0.0|(60,[0,1,2,3,22,3...| 0.03868207103123663|
|  0.0|(60,[0,1,2,11,25,...|0.023845763571790968|


Las regresyjny

In [68]:
forest_reg = regression.RandomForestRegressor()

In [69]:
forest_regModel = forest_reg.fit(df_train)

In [70]:
forest_regModel.featureImportances

SparseVector(60, {0: 0.0116, 1: 0.0074, 2: 0.0214, 3: 0.0159, 4: 0.0018, 5: 0.0061, 6: 0.0003, 7: 0.0137, 8: 0.0462, 9: 0.0019, 10: 0.0292, 12: 0.0003, 13: 0.0007, 14: 0.0011, 15: 0.0027, 21: 0.0379, 22: 0.0085, 23: 0.0017, 24: 0.0084, 25: 0.0014, 26: 0.001, 27: 0.0049, 28: 0.0027, 29: 0.0172, 30: 0.0007, 31: 0.0008, 32: 0.0005, 33: 0.047, 34: 0.0317, 36: 0.0002, 37: 0.0, 38: 0.0029, 40: 0.0029, 41: 0.0004, 43: 0.0008, 57: 0.572, 58: 0.0599, 59: 0.0362})

In [71]:
print(forest_regModel.toDebugString)

RandomForestRegressionModel: uid=RandomForestRegressor_053b97fb8cff, numTrees=20, numFeatures=60
  Tree 0 (weight 1.0):
    If (feature 58 <= 0.5901377405825727)
     If (feature 10 <= 3.7970696388246554)
      If (feature 57 <= 0.01563252463499346)
       If (feature 0 <= 1.0107756116042783)
        If (feature 57 <= 0.009639444538440442)
         Predict: 1.6689847009735743E-4
        Else (feature 57 > 0.009639444538440442)
         Predict: 0.009930486593843098
       Else (feature 0 > 1.0107756116042783)
        If (feature 57 <= 0.009186863662183508)
         Predict: 0.0014905743095133712
        Else (feature 57 > 0.009186863662183508)
         Predict: 0.010655985489721886
      Else (feature 57 > 0.01563252463499346)
       If (feature 3 <= 1.0730935980782343)
        If (feature 57 <= 0.05522359246057472)
         Predict: 0.05126941089474982
        Else (feature 57 > 0.05522359246057472)
         Predict: 0.008012820512820512
       Else (feature 3 > 1.0730935980782343)
  

In [72]:
# predykcje
forest_regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.004155548907145...|
|  0.0|(60,[0,1,2,7,23,2...|0.017855698549755824|
|  0.0|(60,[0,1,2,3,22,3...|0.007681921694254419|
|  0.0|(60,[0,1,2,3,22,3...|0.014467298770185353|
|  0.0|(60,[0,1,2,3,22,3...| 0.08324954986258826|
|  0.0|(60,[0,1,2,15,22,...|0.040750045511460824|
|  0.0|(60,[1,3,22,29,57...| 0.03272717797912787|
|  0.0|(60,[0,1,2,3,22,3...|0.005893582996321674|
|  0.0|(60,[0,2,8,21,24,...| 0.04547931649040385|
|  0.0|(60,[1,2,4,22,29,...|0.013186841537497412|
|  0.0|(60,[0,1,2,4,22,2...|0.009914472497700989|
|  0.0|(60,[1,3,22,28,57...|0.006451478777818283|
|  0.0|(60,[1,3,22,29,57...| 0.00720036267707227|
|  0.0|(60,[0,1,2,9,22,2...|0.015894098486557234|
|  0.0|(60,[1,3,22,36,57...|0.006819538787207068|
|  0.0|(60,[0,1,2,3,22,3...| 0.03481417536209794|
|  0.0|(60,[0,1,2,11,25,...|0.012562098044105782|


Gradient-Boosted Trees regression - Podbicie Gradientowe

In [73]:
gbt_reg = regression.GBTRegressor()

In [74]:
gbt_regModel = gbt_reg.fit(df_train)

In [75]:
gbt_regModel.featureImportances

SparseVector(60, {0: 0.0247, 1: 0.0212, 2: 0.0004, 3: 0.0033, 4: 0.0023, 5: 0.0024, 6: 0.0232, 7: 0.0114, 8: 0.0163, 9: 0.0032, 10: 0.028, 11: 0.0056, 13: 0.0024, 14: 0.009, 15: 0.0056, 16: 0.0029, 17: 0.0005, 18: 0.003, 19: 0.0038, 20: 0.0004, 21: 0.0347, 22: 0.0137, 23: 0.0073, 24: 0.012, 25: 0.0004, 26: 0.0026, 27: 0.0104, 28: 0.0115, 29: 0.0476, 30: 0.0062, 31: 0.0213, 32: 0.0083, 33: 0.0262, 34: 0.0073, 35: 0.0022, 36: 0.0004, 37: 0.0, 38: 0.0129, 39: 0.0102, 40: 0.0052, 41: 0.0077, 42: 0.001, 43: 0.0166, 46: 0.0238, 57: 0.3083, 58: 0.153, 59: 0.0799})

In [76]:
print(gbt_regModel.toDebugString)

GBTRegressionModel: uid=GBTRegressor_66a43bb61e02, numTrees=20, numFeatures=60
  Tree 0 (weight 1.0):
    If (feature 57 <= 0.016074083754750983)
     If (feature 57 <= 0.009311088042728916)
      If (feature 57 <= 0.008275502171638721)
       If (feature 5 <= 2.2628782783944144)
        If (feature 21 <= 1.1993123157773127)
         Predict: 0.0
        Else (feature 21 > 1.1993123157773127)
         Predict: 4.553734061930783E-4
       Else (feature 5 > 2.2628782783944144)
        If (feature 59 <= 0.7446789795243225)
         Predict: 0.0
        Else (feature 59 > 0.7446789795243225)
         Predict: 0.002081165452653486
      Else (feature 57 > 0.008275502171638721)
       If (feature 6 <= 2.7815593385283988)
        If (feature 28 <= 1.006510530627566)
         Predict: 0.0022579026593075764
        Else (feature 28 > 1.006510530627566)
         Predict: 0.004358028830036876
       Else (feature 6 > 2.7815593385283988)
        If (feature 0 <= 1.0107756116042783)
         Predic

In [77]:
# predykcje
gbt_regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.004246753777187538|
|  0.0|(60,[0,1,2,7,23,2...|0.014110238228942765|
|  0.0|(60,[0,1,2,3,22,3...|0.007204597951139047|
|  0.0|(60,[0,1,2,3,22,3...|0.001205467455499...|
|  0.0|(60,[0,1,2,3,22,3...|0.058090266941617105|
|  0.0|(60,[0,1,2,15,22,...| 0.04537353558124911|
|  0.0|(60,[1,3,22,29,57...| 0.03896825039010874|
|  0.0|(60,[0,1,2,3,22,3...|0.003817182331295369|
|  0.0|(60,[0,2,8,21,24,...| 0.05164713774052404|
|  0.0|(60,[1,2,4,22,29,...| 0.01969105475594282|
|  0.0|(60,[0,1,2,4,22,2...|-5.08649060757192...|
|  0.0|(60,[1,3,22,28,57...| 0.00473144484011287|
|  0.0|(60,[1,3,22,29,57...|0.005534043226301918|
|  0.0|(60,[0,1,2,9,22,2...|0.026734343178591562|
|  0.0|(60,[1,3,22,36,57...|0.008834326281026794|
|  0.0|(60,[0,1,2,3,22,3...| 0.04227598453733432|
|  0.0|(60,[0,1,2,11,25,...| 0.02068601578095375|


EWALUACJA

In [78]:
from pyspark.ml import evaluation

In [79]:
evaluator_reg = evaluation.RegressionEvaluator()

In [80]:
evaluator_reg.evaluate(regModel.transform(df_eval))

0.11921036181831389

In [81]:
# rmse - drzewo
evaluator_reg.evaluate(tree_regModel.transform(df_eval))


0.11842982535666526

In [82]:
# rmse - las
evaluator_reg.evaluate(forest_regModel.transform(df_eval))

0.1183558490438668

In [83]:
# rmse - gbt
evaluator_reg.evaluate(gbt_regModel.transform(df_eval))

0.11715871808724128

R^2 powyższych modeli

In [84]:
er2 = evaluation.RegressionEvaluator(metricName="r2")

In [85]:
liniowa = er2.evaluate(regModel.transform(df_eval))
liniowa

0.014131859677981695

In [86]:
drzewo = er2.evaluate(tree_regModel.transform(df_eval))
drzewo

0.026999647787611325

In [87]:
las= er2.evaluate(forest_regModel.transform(df_eval))
las

0.028214823063329675

In [88]:
gbt = er2.evaluate(gbt_regModel.transform(df_eval))
gbt

0.04777398559558799

Próba poprawy R^2

In [98]:
forest_reg2 = regression.RandomForestRegressor(numTrees=45, maxDepth=10)
las2 = er2.evaluate(forest_reg2.fit(df_train).transform(df_eval))
las2

0.08193610949886243

In [99]:
tree_reg2 = regression.DecisionTreeRegressor(maxDepth=5, maxBins=20)
drzewo2 = er2.evaluate(tree_reg2.fit(df_train).transform (df_eval))
drzewo2

0.025189555051740142

In [100]:
gbt_reg2 = regression.GBTRegressor(maxDepth=4, maxBins=20)
gbt2 = er2.evaluate(gbt_reg2.fit(df_train).transform(df_eval))
gbt2

0.03679763200168762

Wybór najlepszych parametrów

In [102]:
from pyspark.ml import tuning
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
#Random Forest

In [103]:
grid = (ParamGridBuilder()
             #.addGrid(rf.maxDepth, [2, 5, 10, 20, 30])
               .addGrid(forest_reg2.maxDepth, [2, 5, 10])
             #.addGrid(rf.maxBins, [10, 20, 40, 80, 100])
               .addGrid(forest_reg2.maxBins, [5, 10, 20])
             #.addGrid(rf.numTrees, [5, 20, 50, 100, 500])
               .addGrid(forest_reg2.numTrees, [5, 20, 50])
             .build())

In [122]:
reg_eval = evaluation.RegressionEvaluator(metricName='r2')

In [123]:
cv_las = tuning.CrossValidator(estimator=forest_reg2, estimatorParamMaps=grid, evaluator=reg_eval)

In [124]:
cvModel_las = cv_las.fit(df_train)

In [125]:
cvModel_las.avgMetrics

[0.014735353739925738,
 0.013186293163274614,
 0.013567051444551245,
 0.01724956481204476,
 0.014325957498391198,
 0.014829121985658927,
 0.01732053310207234,
 0.0143159603120722,
 0.014823593328240275,
 0.019742655840989592,
 0.02046104414241227,
 0.020403578096347408,
 0.021280821459726245,
 0.022033341840819582,
 0.022274408256024524,
 0.02218691205705899,
 0.02156848444315529,
 0.022545933543245078,
 0.013911029277778247,
 0.0191162596134918,
 0.020318321617457058,
 0.011562005914866854,
 0.020156701233493075,
 0.02160440797971752,
 0.015914866391287842,
 0.019523272878286552,
 0.022928117321778907]

In [ ]:
#Drzewo regresyjne

In [131]:
grid_drzewo = (ParamGridBuilder()
               .addGrid(tree_reg2.maxDepth, [2, 5, 10])
               .addGrid(tree_reg2.maxBins, [5, 20, 50])
             .build())

In [132]:
cv_drzewo = tuning.CrossValidator(estimator=tree_reg2, estimatorParamMaps=grid_drzewo, evaluator=reg_eval)

In [133]:
cvModel_drzewo = cv_drzewo.fit(df_train)

In [134]:
cvModel_drzewo.avgMetrics

[0.015284903888714212,
 0.014851718863248764,
 0.01657232306329237,
 0.017582922532674055,
 0.017764741416565588,
 0.017584251492814208,
 -0.034370768815812656,
 -0.05175157053114954,
 -0.07468071201821595]

In [ ]:
#GBT

In [155]:
grid_gbt = (ParamGridBuilder()
             .addGrid(gbt_reg2.maxDepth, [2, 5, 10])
             .addGrid(gbt_reg2.maxBins, [10, 20, 40])
             .addGrid(gbt_reg2.maxIter, [5, 10, 20])
             .build())

In [156]:
gbevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="rmse")

In [157]:
gbcv = CrossValidator(estimator = gbt_reg2,
                      estimatorParamMaps = grid_gbt,
                      evaluator = gbevaluator,
                      numFolds = 5)

In [159]:
cvModel_gbt = gbcv.fit(df_train)

In [160]:
cvModel_gbt.avgMetrics

[0.11887382698837876,
 0.11876525429599019,
 0.11866161031592637,
 0.11887458610374486,
 0.11876347546469554,
 0.11865891267774928,
 0.1188616689928584,
 0.11874008363423061,
 0.11859684858516772,
 0.11885550128091367,
 0.11887245584899431,
 0.11888182015849993,
 0.11879149153893767,
 0.11874409368690034,
 0.11874794473254005,
 0.11876079146131643,
 0.11869784944958492,
 0.11869112135978893,
 0.12162336220862116,
 0.12210339554751097,
 0.1229730815322812,
 0.12185447201480504,
 0.12226367016209103,
 0.12322681205260555,
 0.12336309161412301,
 0.12383879786747759,
 0.1251077430450466]

Parametry najlepszego modelu

In [126]:
cvModel_las.bestModel._java_obj.getMaxDepth()

10

In [127]:
cvModel_las.bestModel._java_obj.getMaxBins()

20

In [128]:
cvModel_las.bestModel._java_obj.numFeatures()

60

In [129]:
cvModel_las.bestModel._java_obj.numTrees()

JavaObject id=o41512

In [113]:
forest_reg3 = regression.RandomForestRegressor(numTrees=60, maxDepth=10)
las3 = er2.evaluate(forest_reg3.fit(df_train).transform(df_eval))

In [114]:
las3

0.08511618022451428

In [150]:
cvModel_drzewo.bestModel._java_obj.getMaxDepth()

5

In [151]:
cvModel_drzewo.bestModel._java_obj.getMaxBins()

20

In [153]:
tree_reg3 = regression.RandomForestRegressor(maxDepth=5, maxBins=20)
drzewo3 = er2.evaluate(tree_reg3.fit(df_train).transform(df_eval))

In [154]:
drzewo3

0.02774345967658509

In [161]:
cvModel_gbt.bestModel._java_obj.getMaxDepth()

2

In [162]:
cvModel_gbt.bestModel._java_obj.getMaxBins()

40

In [163]:
gbt_reg3 = regression.GBTRegressor(maxDepth=2, maxBins=40)
gbt3 = er2.evaluate(gbt_reg3.fit(df_train).transform(df_eval))
gbt3

0.02515029657264578

# ***WYNIKI***

In [164]:
wynik = spark.createDataFrame([Row(metoda='Drzewo regresyjne', rSquared=drzewo),
                               Row(metoda='Las regresyjny', rSquared=las),
                               Row(metoda='GBT', rSquared=gbt)])
wynik_r = spark.createDataFrame([Row(metoda='Drzewo regresyjne', rSquared_poprawione=drzewo2),
                               Row(metoda='Las regresyjny', rSquared_poprawione=las2),
                               Row(metoda='GBT', rSquared_poprawione=gbt2)])
wynik_best = spark.createDataFrame([Row(metoda='Drzewo regresyjne', rSquared_best=drzewo3),
                               Row(metoda='Las regresyjny', rSquared_best=las3),
                               Row(metoda='GBT', rSquared_best=gbt3)])
wynik.show()
wynik_r.show()
wynik_best.show()

+-----------------+--------------------+
|           metoda|            rSquared|
+-----------------+--------------------+
|Drzewo regresyjne|0.026999647787611325|
|   Las regresyjny|0.028214823063329675|
|              GBT| 0.04777398559558799|
+-----------------+--------------------+

+-----------------+--------------------+
|           metoda| rSquared_poprawione|
+-----------------+--------------------+
|Drzewo regresyjne|0.025189555051740142|
|   Las regresyjny| 0.08193610949886243|
|              GBT| 0.03679763200168762|
+-----------------+--------------------+

+-----------------+-------------------+
|           metoda|      rSquared_best|
+-----------------+-------------------+
|Drzewo regresyjne|0.02774345967658509|
|   Las regresyjny|0.08511618022451428|
|              GBT|0.02515029657264578|
+-----------------+-------------------+



In [ ]:
#wynik.toPandas()